In [1]:
'''
reading all libraries and packages that we need
'''
import os
import sys
import mrc
import time
import pickle
import numpy as np
import pandas as pd
from skimage import io
import SimpleITK as sitk
from matplotlib import pyplot as plt
from skimage.feature import peak_local_max
from skimage.segmentation import mark_boundaries,clear_border, watershed
from skimage.measure import regionprops, label
from scipy.ndimage import binary_fill_holes
from scipy import ndimage as ndi
from skimage.morphology import closing, square, remove_small_objects, binary_erosion, disk, binary_dilation, convex_hull_image
#from skimage.color import label2rgb
from skimage.filters import  threshold_otsu, threshold_triangle, gaussian, threshold_local
from skimage.draw import line, polygon
import multiprocessing as mp
from multiprocessing import Pool
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

In [2]:
def prune_and_label_dapi(seg_dapi, debris_size, erosion_radius):
    ''' Segments the dapi image.
    Ipnuts:
    
    seg_dapi: 2-dimensional numpy array (segmented image array)
    debris_size: size of the debris to be removed (in pixels)
    erosion_radius: radius (in pixels) of the disk to be used for binary ersion to separate connected nuclei
    
    Outputs:
    nuclear_mask: segmented nuclei image
    nuclear_labels: labled nuclei
    
    '''
    # Step 1: Remove small debris from the segmented image using the specified debris size threshold
    seg_dapi_1 = remove_small_objects(seg_dapi, debris_size)

    # Step 2: Erode the image slightly using a disk-shaped structuring element (disk size 2) to disconnect close objects
    seg_dapi_2 = binary_erosion(seg_dapi_1, disk(2))
    
    # Step 3: Fill any holes in the eroded image to create solid nuclei regions
    nuclear_mask = ndi.binary_fill_holes(seg_dapi_2)
    
    # Step 4: Further erode the nuclear mask using a disk with the specified erosion radius
    # This helps in separating nuclei that are too close together
    eroded_mask = binary_erosion(nuclear_mask, disk(erosion_radius))

    # Step 5: Perform distance transform to compute the Euclidean distance to the nearest background pixel
    distance = ndi.distance_transform_edt(eroded_mask)

    # Step 6: Find local maxima in the distance-transformed image, which will serve as markers for the watershed
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 1)),
                                labels=nuclear_mask)

    # Step 7: Label the local maxima to create markers
    markers = ndi.label(local_maxi)[0]

    # Step 8: Apply the watershed algorithm using the negative distance transform and the markers
    # This will segment the nuclei by separating connected regions based on the distance map
    nuclear_labels = watershed(-distance, markers, mask=nuclear_mask, connectivity=2)
    
    # Return the segmented mask and the labeled nuclei
    return nuclear_mask, nuclear_labels

In [3]:
def segment_and_label_dapi(dapi_image, debris_size, min_area_to_keep_cell, erosion_radius, block_size = 251):
    ''' Segments the dapi image.
    Ipnuts:
    
    dapi_image: 2-dimensional numpy array (image array)
    debris_size: size of the debris to be removed (in pixels)
    erosion_radius: radius (in pixels) of the disk to be used for binary ersion to separate connected nuclei
    
    Outputs:
    nuclear_mask: segmented nuclei image
    nuclear_labels: labled nuclei
    
    '''
    # Step 1: Perform local adaptive thresholding using a block size to binarize the DAPI image
    # Adaptive thresholding adjusts the threshold for small regions, useful for uneven illumination
    adaptive_thresh = threshold_local(dapi_image, block_size = block_size)

    # Step 2: Create a binary mask where the DAPI image is greater than the adaptive threshold
    seg_dapi = dapi_image > adaptive_thresh
    

    # Step 3: Fill any small holes in the binary mask to create solid nuclei regions
    nuclear_mask = ndi.binary_fill_holes(seg_dapi)
    
    # Step 4: Remove small debris from the segmented image based on the debris size threshold
    seg_dapi = remove_small_objects(seg_dapi, debris_size)

    
    # Step 5: Remove any small nuclei that are below the minimum area threshold
    nuclear_mask = remove_small_objects(nuclear_mask, min_area_to_keep_cell)
    
    # Step 6: Erode the image using a disk-shaped structuring element (with the specified erosion radius)
    # This helps in separating nuclei that are close together
    eroded_mask = binary_erosion(nuclear_mask, disk(erosion_radius))

    # Step 7: Compute the Euclidean distance transform to measure the distance to the nearest background pixel
    distance = ndi.distance_transform_edt(eroded_mask)

    # Step 8: Identify local maxima in the distance-transformed image, which will be used as markers for watershed
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 1)),
                                labels=nuclear_mask)

    # Step 9: Label the local maxima to generate markers for the watershed algorithm
    markers = ndi.label(local_maxi)[0]

    # Step 10: Apply the watershed algorithm to segment the nuclei based on the distance transform
    # This separates connected regions using the markers as starting points
    nuclear_labels = watershed(-distance, markers, mask=nuclear_mask, connectivity=2)

    # Return the final segmented mask and labeled nuclei
    return nuclear_mask, nuclear_labels

In [4]:
def obtain_big_cells(labeled_img, area_thresh, cir_thresh):
    ''' Determining the package of the cells as a big cell.
    Ipnuts:
    
    labeled_img: nuclei label from the labling function
    area_thresh: maximum size of a nucleus (in pixels) to be considered as a single nucleus
    cir_thresh: circularity threshold to see whether it is a single nucleus or multiple nuclei
    
    Outputs:
    big_cells: label of the cells that are satistfying the area AND circularity threshold
    '''
    # Lambda function to calculate the circularity of a nucleus
    # Circularity is calculated as (4 * π * Area) / Perimeter^2
    # This gives a value between 0 and 1, with values closer to 1 indicating more circular shapes
    
    circ = lambda r: (4 * np.pi * r.area) / (r.perimeter * r.perimeter)

    # List comprehension to find nuclei that satisfy both area and circularity thresholds
    big_cells = [(prop.label, prop.area, circ(prop)) # For each region, store the label, area, and circularity
                 for prop in regionprops(labeled_img) # Iterate through each labeled region (nucleus)
                 if prop.area > area_thresh and circ(prop) < cir_thresh] # Apply the area and circularity filters

    # Return the list of big cells that meet the criteria
    return big_cells

In [5]:
def split_cells(label_rem, image):
    ''' Splitting a package of the cells
    Ipnuts:
    
    labeled_rem: label of the object that should be splitted
    image: image: 2D labeled image
    
    Outputs:
    labels_rem: label of the cells that are going to split
    
    '''
    # Initialize arrays to store relabeled and temporary images
    relabels, tmp_img = np.zeros_like(image), np.zeros_like(image)

    # Initialize lists to store points and coordinates of centroids
    pts = []
    row_cords =[]
    col_cords = []

    # Step 1: Loop through each region in the labeled object to extract its centroid coordinates
    for prop in regionprops(label_rem):
        x,y = np.int16(np.round(prop.centroid)) # Get the centroid coordinates (rounded to integer)
        row_cords.append(x) # Store row coordinates
        col_cords.append(y) # Store column coordinates
        pts.extend([x,y]) # Add the centroid coordinates to the points list

    # Step 2: If there are more than 2 regions, draw a polygon around the centroids
    if len(regionprops(label_rem))>2:
        rr, cc = polygon(row_cords, col_cords) # Create a polygon connecting the centroids
        
    else:
        # Otherwise, draw a straight line between two centroids
        rr, cc = line(pts[0], pts[1], pts[2], pts[3])

    # Step 3: Mark the region inside the polygon/line in the temporary image
    tmp_img[rr,cc] =1
    
    # Step 4: Dilate the binary line/polygon to cover more area
    tmp_img = binary_dilation(tmp_img, disk(10))

    # Step 5: Compute the convex hull of the dilated image to create a filled area
    tmp_img = convex_hull_image(tmp_img)

    # Step 6: Create a mask for the regions that are not covered by the convex hull
    split_img = np.logical_and(image, np.logical_not(tmp_img))

    # Step 7: Perform distance transform to compute the distance of each pixel to the nearest background
    distance_rem = ndi.distance_transform_edt(split_img)

    # Step 8: Find local maxima in the distance-transformed image, which will act as markers for watershed
    local_maxi_rem = peak_local_max(distance_rem, indices=False, footprint=np.ones((1, 1)),
                                labels=image)

    # Step 9: Label the local maxima as markers for watershed segmentation
    markers_rem = label(local_maxi_rem, connectivity=2)

    # Step 10: Apply the watershed algorithm to split connected regions based on the distance map
    labels_rem = watershed(-distance_rem, markers_rem, mask=image, connectivity=2)

    # Return the relabeled image after splitting
    return labels_rem

In [6]:
def obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, 
                                                    ero_rad, min_rem_area, min_area_to_keep_cell):
    '''
    Inputs:
    nuclear_labels: 2D numpy labeled dapi image
    big_cells: label of the big objects
    ero_rad: radius of the disk to be used for eroading the region (convex hull - roi)
    min_rem_area: removing any object less than of this size (in pixels)
    min_area_to_keep_cell: minimum size of an acceptable nucleus (in mpixels)
    
    Outputs:
    final_label: final label of nuecleus after removing small objects and splitting big packages
    '''

    # Initialize an array to hold the relabeled image after splitting large objects
    relabels = np.zeros_like(nuclear_labels)

    # Step 1: Loop through each big cell that needs to be split
    for ii, area, cir in big_cells:
        image = nuclear_labels==ii # Create a binary mask for the current big cell

        # Step 2: Generate the convex hull of the current cell to form a tight boundary around it
        chull = convex_hull_image(image)

        # Step 3: Identify the region outside the original cell but within the convex hull
        rem = np.logical_and(chull, np.logical_not(image))

        # Step 4: Erode this external region to shrink it slightly and separate connected objects
        eroded_rem = binary_erosion(rem, disk(ero_rad))

        # Step 5: Remove small objects from the eroded region based on the minimum remaining area threshold
        eroded_rem = remove_small_objects(eroded_rem, min_rem_area)

        # Step 6: Label the remaining eroded region to identify individual components
        label_rem = label(eroded_rem)

        # Step 7: If there are two or more connected components in the labeled region, split them
        if np.max(label_rem) >= 2:
            # Update the relabeled image with the split cells from the large object
            relabels = label(relabels + split_cells(label_rem, image))## split cells

    # Step 8: Initialize an array to hold the newly assigned labels after splitting
    assigned_relabels = np.zeros_like(relabels)

    # Step 9: Assign new labels to the split regions, ensuring no overlap with existing labels
    for p in regionprops(relabels):
        if p.label > 0:
            # Assign a new label to each split region that doesn't overlap with existing labels
            assigned_relabels[np.where(relabels==p.label)] = np.max(nuclear_labels) + p.label

    # Step 10: Add the original nuclear labels and the newly assigned relabeled regions
    label_img = label(nuclear_labels + assigned_relabels)
    
    # Step 11: Remove small objects from the final labeled image based on the minimum area threshold for a valid nucleus
    label_img = remove_small_objects(label_img, min_area_to_keep_cell)

    # Final relabeled image after splitting and filtering
    final_labels = label_img
    return final_labels

In [7]:
def create_dir(dir_name):
    # Check if the directory does not exist
    if not os.path.exists(dir_name):
            # If the directory does not exist, create it
            os.makedirs(dir_name)
    return None

In [8]:
root_path = r"D:\12-January21\GREB1\FVWAE2"

In [9]:
fpaths = [os.path.join(root_path,f) for f in os.listdir(root_path) if f.endswith('.dv')]

In [10]:
# Default Parameters for Image Segmentation and Processing
sigma = 1 # Standard deviation for Gaussian filtering, typically used for smoothing or noise reduction
erosion_radius = 31 # Radius (in pixels) of the disk used for morphological erosion; larger values lead to more aggressive erosion
debris_size = 100 # Minimum size (in pixels) for debris to be removed during preprocessing; any object smaller than this will be considered debris
min_area_to_keep_cell = 3000 # Minimum size (in pixels) of an acceptable nucleus; nuclei smaller than this size will be filtered out
block_size = 651 # Block size for adaptive thresholding; defines the local neighborhood size used to calculate the threshold for segmentation
dilation_radius = 100 # Radius (in pixels) for morphological dilation; larger values expand the object boundaries during post-processing
area_thresh =20000 # Maximum area (in pixels) for a nucleus to be considered a single cell; objects larger than this will likely be split
cir_thresh = 0.70 # Circularity threshold; objects with circularity below this value will be considered for splitting, as they are likely not single cells
ero_rad = 5 # Radius (in pixels) of the disk used for eroding the convex hull region; smaller values make more subtle changes
min_rem_area = 2 # Minimum remaining area (in pixels) after erosion for the region to be kept; smaller objects are removed

In [11]:
# Create a directory name that includes specific parameters in the name
seg_dir = os.path.join(root_path, 'Segmentation_erosion_radius_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh))

# Check if the directory already exists
if not os.path.exists(seg_dir):
    # If the directory does not exist, create it
    os.makedirs(seg_dir)

In [12]:
# Define the path for the directory to store raw images
raw_img_dir = os.path.join(os.path.dirname(seg_dir), 'raw')
# Define the path for the directory to store raw images with top and bottom cropped
raw_img_dir_leave_top_bottom = os.path.join(os.path.dirname(seg_dir), 'raw_projected')
# Define the path for the directory to store segmented DAPI images
seg_dapi_dir = os.path.join(os.path.dirname(seg_dir), 'seg_dapi')
# List of directories to create if they don't already exist
for dir_name in [raw_img_dir, raw_img_dir_leave_top_bottom, seg_dapi_dir]:
    # Check if the directory does not exist
    if not os.path.exists(dir_name):
        # Create the directory
        os.makedirs(dir_name)


In [13]:
# Initialize an empty DataFrame to store features
features = pd.DataFrame()
# Iterate over each file path in the list 'fpaths'
for fpath in fpaths:
    # Extract the file name from the path
    fname = os.path.basename(fpath)
    # Create a main file name by joining parts of the file name with underscores
    main_fname = ('_').join(fname.split('_')[0:])
    # Check if 'wash' is present in the file name
    if 'wash' in fname:
        # Remove the file extension from the file name
        fname_noext = os.path.splitext(fname)[0]
        # Split the file name into components
        year, mon_date,  gene,_,_,_,time,field, _, _ =fname_noext.split('_')
        # Append a new row to the DataFrame with extracted features
        features = features.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'gene': gene,
                                         'time': time,
                                         'field' : field,
                                          },])
    else:
        # Remove the file extension from the file name
        fname_noext = os.path.splitext(fname)[0]
        # Split the file name into components
        year, mon_date, gene,_,field,_,_ =fname_noext.split('_')
        # Append a new row to the DataFrame with extracted features
        features = features.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'gene': gene,
                                         'time': '0',
                                         'field' : field,
                                          },])
# Reset the DataFrame index after appending rows
features = features.reset_index(drop=True)

In [14]:
def image_projection(path,df):
        """
    Process an image file by performing the following steps:
    1. Read the image and compute projections for different channels.
    2. Save the projections to specified directories.
    3. Perform segmentation and labeling on the DAPI channel.
    4. Identify and segment large cells and refine labels.
    5. Mark boundaries of nuclei and save the results.

    Parameters:
    - path: str
      The file path to the image to be processed.
    - df: pandas.DataFrame
      A DataFrame containing metadata about the images. Used to filter for the current image.

    Returns:
    None
    """
    # Extract the base name of the file and change the extension to '.tif'
    fname = os.path.splitext(os.path.basename(path))[0] + '.tif'

    # Filter the DataFrame to get the row corresponding to the given file path
    df = df[df['filepath'] == path]

    # Read the image file into an array
    img_array = mrc.imread(path)

    # Get the dimension size of the image
    DIM=img_array.shape[1]

    # Compute the maximum projection along the z-axis for each channel and save the results
    # Projection for DAPI channel (all slices)
    dapi_image = np.max(img_array[0], axis=0)
    io.imsave(os.path.join(raw_img_dir, 'dapi_' + fname), dapi_image)
    
    # Projection for DAPI channel (excluding top and bottom slices)
    dapi_image = np.max(img_array[0, 2:DIM-1], axis=0)
    io.imsave(os.path.join(raw_img_dir_leave_top_bottom, 'dapi_' + fname), dapi_image)
    
    # Projection for Exon channel (all slices)
    exon_image = np.max(img_array[1], axis=0)
    io.imsave(os.path.join(raw_img_dir,'exon_' + fname), exon_image)
    
    # Projection for Exon channel (excluding top and bottom slices)
    exon_image = np.max(img_array[1, 2:DIM-1], axis=0)
    io.imsave(os.path.join(raw_img_dir_leave_top_bottom,'exon_' + fname), exon_image)
    
    # Projection for Intron channel (all slices)
    intron_image = np.max(img_array[2], axis=0)
    io.imsave(os.path.join(raw_img_dir,'intron_' + fname), intron_image)
    
    # Projection for Intron channel (excluding top and bottom slices)
    intron_image = np.max(img_array[2, 2:DIM-1], axis=0)
    io.imsave(os.path.join(raw_img_dir_leave_top_bottom,'intron_' + fname), intron_image)
    
    # Segment and label nuclei in the DAPI image
    nuclear_mask, nuclear_labels = segment_and_label_dapi(dapi_image, debris_size, 
                                                          min_area_to_keep_cell,
                                                          erosion_radius,
                                                         block_size = block_size)

    # Identify and segment big cells in the nuclear labels
    big_cells = obtain_big_cells(nuclear_labels, area_thresh, cir_thresh)

    # Obtain final labels after splitting big cells and additional processing
    final_labels = obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, ero_rad, 
                                                min_rem_area, min_area_to_keep_cell)
    # Clear labels of nuclei touching the border
    non_border_labels = clear_border(final_labels)
    # Create a mask for nuclei
    nuc_mask = final_labels > 0
    # Mark boundaries of nuclei in the DAPI image
    marked_dapi = mark_boundaries(dapi_image, non_border_labels, color=(1, 1, 1), outline_color=(1, 1, 1))
    # Save the marked DAPI image with boundaries highlighted
    io.imsave(os.path.join(seg_dapi_dir, 'dapi_' + fname), np.uint8(marked_dapi*255))
    

In [15]:
# Parallelize the execution of the image_projection function across multiple files
# - n_jobs=8: Specifies that 8 parallel jobs (or threads) will be used.
# - delayed: Wraps the image_projection function to delay its execution until it's ready to be parallelized.
# - features['filepath'].unique(): Retrieves the unique file paths from the 'features' DataFrame, so that the function is applied to each unique path.

project=Parallel(n_jobs=8)(delayed(image_projection)(path,features) for path in features['filepath'].unique())

In [16]:
# Create a list of full file paths for all TIFF files in the directory `raw_img_dir_leave_top_bottom`
# Generate the full path by joining the directory path with each file name
# Iterate over all files in the specified directory
# Include only files that end with the '.tif' extension

newfpaths = [os.path.join(raw_img_dir_leave_top_bottom,f) for f in os.listdir(raw_img_dir_leave_top_bottom) if f.endswith('.tif')]

In [17]:
#Segmenting DAPI channel
# Define the alpha parameter for sharpening

alpha = 40
# Iterate over each file path in the list of new TIFF files
for fpath in newfpaths:
    # Process only the files that have 'dapi' in their file path
    if 'dapi' in fpath:
        
        # Read the DAPI image from the file path
        dapi_image = io.imread(fpath)

        # Create a local variable for convenience
        f = dapi_image
        
        # Apply Gaussian blur to the image with a sigma of 3
        blurred_f = ndi.gaussian_filter(f, 3)

        # Apply a second Gaussian blur with a smaller sigma of 1
        filter_blurred_f = ndi.gaussian_filter(blurred_f, 1)

        # Sharpen the image by subtracting the blurred image from the original and scaling
        sharpened = blurred_f + alpha * (blurred_f - filter_blurred_f)

        # Apply a binary threshold to the DAPI image to fill holes
        filled_seg_dapi = ndi.binary_fill_holes(dapi_image > (0.6*threshold_otsu(dapi_image)))

        # Compute the local adaptive threshold for the sharpened image
        adaptive_thresh = threshold_local(sharpened, block_size = block_size)

        # Apply the local adaptive threshold to the sharpened image and fill holes
        temp_seg_2 = ndi.binary_fill_holes (sharpened > adaptive_thresh)
        
        # Combine the binary masks from the DAPI thresholding and adaptive thresholding
        seg_dapi = np.logical_and(filled_seg_dapi, temp_seg_2)

        # Extract the base name of the file
        fname = os.path.basename(fpath)

        # Save the segmented DAPI image to the specified directory
        io.imsave(os.path.join(seg_dapi_dir, 'seg_' + fname), np.uint8(255*seg_dapi))       

In [18]:
# Initialize an empty DataFrame to store new features
newfeatures = pd.DataFrame()

# Iterate over each file path in the list of new TIFF files
for fpath in newfpaths:
    # Extract the base name of the file
    fname = os.path.basename(fpath)
     # Create a main file name by joining parts of the file name with underscores
    main_fname = ('_').join(fname.split('_')[1:])
    # Process files containing 'wash' in their name
    if 'wash' in fname:
        # Remove the file extension from the file name
        fname_noext = os.path.splitext(fname)[0]
        # Split the file name into components
        wavelength, year, mon_date, _,_,_,_,time,field, _, _ =fname_noext.split('_')
        # Append the extracted features to the DataFrame
        newfeatures = newfeatures.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'time': time,
                                         'field' : field,
                                     'wavelength' : wavelength,
                                          },])
    else:
        # Remove the file extension from the file name
        fname_noext = os.path.splitext(fname)[0]
         # Split the file name into components
        wavelength, year, mon_date, _,_, field, _, _ =fname_noext.split('_')
        # Append the extracted features to the DataFrame with a default time value
        newfeatures = newfeatures.append([{'filename': main_fname,
                                         'filepath': fpath,
                                         'date': ('').join([year,mon_date]),
                                         'time': '0',
                                         'field' : field,
                                     'wavelength' : wavelength,
                                          },])
# Reset the DataFrame index after appending rows
newfeatures = newfeatures.reset_index(drop=True)
# Convert the 'field' column to integer type
newfeatures = newfeatures.astype({"field": int})


In [19]:
# Define a dictionary to map time strings to numerical values
t = {'15min': 15, '30min':30, '45min': 45, '60min': 60, '75min': 75, '90min':90,'0':0}
# Map the 'time' column in the DataFrame to the corresponding numerical values using the dictionary
newfeatures["time"] = newfeatures["time"].map(t)

In [20]:
# Construct a directory path for storing segmentation results, incorporating key parameters into the folder name
seg_dir = os.path.join(root_path,  'Segmentation_erosion_radius_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh))

# Check if the directory already exists, and if not, create it
if not os.path.exists(seg_dir):
    os.makedirs(seg_dir)

In [21]:
def image_segmentation(filename,feat):
        """
    This function segments and analyzes images based on different channels (DAPI, exon, intron).
    It performs segmentation on nuclei (DAPI), exonic spots, and intronic spots, and then checks 
    for colocalization between exonic and intronic spots to identify nascent mRNA. The results are
    saved into a summary dictionary, and finally saved to a pickle file.
    
    Args:
        filename (str): The name of the file to process.
        feat (DataFrame): DataFrame containing features including file paths, time, field, and wavelength.

    Returns:
        None. (Saves the segmentation summary to a pickle file.)
    """
    summary = {}

    # Filter the DataFrame for the specific file being processed
    df = feat[feat['filename'] == filename]
    t=df.iloc[0]['time']#str(df.iloc[0]['time']) + "_" + df.iloc[0]['treatment']
    f=df.iloc[0]['field']
    
    # Initialize summary dictionary for this time and field
    summary['time' +str(t)+'field'+str(f)]={}
    
    # Loop over each channel (wavelength) in the DataFrame for the current file
    for channel in df['wavelength']:
        sdf = df[df['wavelength']==channel]
        fname = os.path.basename(list(sdf['filepath'])[0])
        
        ## Segmenting the nuclei in the DAPI channel
        if channel == 'dapi':
            seg_name = 'seg_' + fname
            seg_dapi =  io.imread(os.path.join(seg_dapi_dir, seg_name))
            
            # Prune and label the DAPI channel to get nuclear labels
            nuclear_mask, nuclear_labels = prune_and_label_dapi(seg_dapi, debris_size, erosion_radius)

            # Obtain big cells that may need splitting if they contain more than one nucleus
            big_cells = obtain_big_cells(nuclear_labels, area_thresh, cir_thresh)

            # Split large objects and obtain final nuclear labels
            final_labels_1 = obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, ero_rad, 
                                                                min_rem_area, min_area_to_keep_cell)

            nuc_mask = final_labels_1

            # Dilate the nuclear mask slightly to ensure full coverage
            temp_img = np.logical_or(binary_dilation(nuc_mask, disk(2)), binary_fill_holes(seg_dapi))

            # Use the watershed algorithm to segment cells based on nuclear labels
            final_labels = watershed(temp_img, markers=final_labels_1, mask=temp_img)

            # Remove labels of nuclei that touch the border of the image
            non_border_labels = clear_border(final_labels)
            

        ## Segmenting the exonic spots in the exon channel
        elif channel == 'exon':
            exon_image =  io.imread(os.path.join(fpath, fname))
            seg_exon = exon_image > (threshold_otsu(exon_image))
            
        
        ## Segmenting the intronic spots in the intron channel
        elif channel == 'intron':
            intron_image = sitk.ReadImage(os.path.join(fpath, fname))
            gaussian_blur = sitk.SmoothingRecursiveGaussianImageFilter()
            gaussian_blur.SetSigma ( float ( sigma ) )
            blur_intron = gaussian_blur.Execute ( intron_image )

            # Apply Maximum Entropy Thresholding for segmentation
            max_entropy_filter = sitk.MaximumEntropyThresholdImageFilter()
            max_entropy_filter.SetInsideValue(0)
            max_entropy_filter.SetOutsideValue(1)
            seg = max_entropy_filter.Execute(blur_intron)
            seg_intron = sitk.GetArrayFromImage(seg)

            # Determine threshold for final segmentation
            blur_intron_img = sitk.GetArrayFromImage(blur_intron)
            intron_threshold = np.min(blur_intron_img[np.where(seg_intron != 0)])
            seg_intron = blur_intron_img > intron_threshold
            
        else:
            print('Different than dapi, exon or intron image found!')
            
    ## Saving the segmentation of all three channels as a combined image                
    comb_img = np.zeros((seg_exon.shape[0], seg_exon.shape[0],3)) # Initialize combined image
    comb_img[:,:,0] = seg_intron # Red channel for intron
    comb_img[:,:,1] = seg_exon # Green channel for exon
    comb_img[:,:,2] = nuc_mask # Blue channel for DAPI (nuclei)

    # Mark boundaries of nuclei and save the combined image
    marked_img = mark_boundaries(comb_img, non_border_labels, color=(1, 1, 1), outline_color=(1, 1, 1))
    io.imsave(os.path.join(seg_dir, 'combined_' + fname), np.uint8(marked_img*255))
    
    
    ## Label the segmented spots in all channels    
    green=label(seg_exon) # Label exonic spots
    red=label(seg_intron) # Label intronic spots
    blue=label(non_border_labels) # Label DAPI (nuclei)
        


    ### Extract measurements for each nucleus
    
    ## First, for each nucleus within the DAPI channel we find the exonic spots that are inside that nucleus

    for nuc in regionprops(blue):
        nuc_id=nuc.label # Get the nucleus ID
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]={}
        # Initialize lists to store spot data for each nucleus
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['intron']=[]
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon']=[]
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot']=[]
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Ncolocspot']=[]
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent']=[]
        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Nnascent']=[]

        # Identify exonic spots that are inside the nucleus
        for exon in regionprops(green,intensity_image=exon_image):
            exon_id=exon.label
            if (((exon.coords[:, None] == nuc.coords).all(-1).any(-1)==True).any()):
                summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon'].append(exon_id)
                
        ## Check for colocalization between exonic and intronic spots (nascent mRNA)
        for intron in regionprops(red,intensity_image=sitk.GetArrayFromImage(intron_image)):
            intron_id=intron.label
            if (((intron.coords[:, None] == nuc.coords).all(-1).any(-1)==True).all()):
                summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['intron'].append(intron_id)         

                # Check for colocalization between exonic and intronic spots
                for nucexon in summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon']:
                    if (((intron.coords[:, None] == regionprops(green)[nucexon-1].coords).all(-1).any(-1)==True).any()):
                        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot'].append([intron_id,nucexon])
                        
        ## Finally, on some locations, one intronic spot has colocation with two (or more) exonic spots or
        ## one exonic spots has colocation with two (or more) intronic spots.
        ## we are going to set them all equally and consider them just as "one nascent mRNA" or "one burst"
        ## Handle cases where colocalization involves multiple exonic or intronic spots
        X=np.array(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot'])
        W= np.array(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['intron'])
        Y=np.array(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['exon'])

        if(len(X)>0):
            # Check if there are overlapping spots and adjust to count them as one nascent mRNA
            if((len(X[:,0])>len(np.unique(X[:,0]))) or(len(X[:,1])>len(np.unique(X[:,1])))) :
                unique, counts = np.unique(X[:,0], return_counts=True)
                unique2, counts2 = np.unique(X[:,1], return_counts=True)
                if((len(X[:,0])>len(np.unique(X[:,0])))):
                    for i in (unique[(counts)>1]):
                        # Merge non-overlapping and overlapping spots into the nascent array
                        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'].append([i,X[X[:,0]==i][:,1]])
                    v=([X[X[:,0]==i][0][0] for i in unique[(counts)==1]])
                    u=([X[X[:,0]==i][0][1] for i in unique[(counts)==1]])
                    r=([X[X[:,0]==i][0] for i in unique[(counts)==1]])
                    if (len(u)>len(np.unique(u))):
                        for j in np.unique(u):
                            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'].append([X[X[:,1]==j][:,0],j])
                    elif(len(r)>0):
                        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent']=np.vstack((summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'],r))
                else:
                    # Handle overlapping exonic spots
                    for k in (unique2):
                        summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'].append([X[X[:,1]==k][:,0],k]) 
            else:
                # If there are no overlaps, nascent mRNA is the same as colocalized spots
                summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent']=summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['colocspot']
        else:
            # If no nascent spots are found, store the count of nascent mRNAs
            summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['Nnascent']=len(summary['time' +str(t)+'field'+str(f)]['nuc'+str(nuc_id)]['nascent'])   
    ## Save the segmentation summary as a pickle file
    with open(os.path.join(seg_dir, 'summary_t_'+str(t)+'_f_'+str(f)+'.pickle'), 'wb') as handle:
        pickle.dump(summary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
fpath = raw_img_dir_leave_top_bottom
from time import time
begin=time()

Parallel(n_jobs=2)(delayed(image_segmentation)(filename,newfeatures) for filename in newfeatures['filename'].unique())
        
print("Total time:",time()-begin)

Total time: 34455.50477766991
